## 0. Import libraries and define functions

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import timedelta
from datetime import datetime
from datetime import date
import warnings
warnings.filterwarnings('ignore')

def only_duration(dur):
        duration = re.findall("\d",dur)
        duration = ''.join(duration)
        return duration

## 1. Define both website and elements to scrape

In [2]:
#####--- 1.0 Read Config_File and set some variables
yaml        = YAML(typ='safe')
with open('Config_File.yml', 'r') as fil:
        config = yaml.load(fil)
config_general = config["general"]
scrape_pgr_class = config_general["scrape_pgr_class"]
scrape_type_dur = config_general["scrape_type_dur"]
scrape_start_class = config_general["scrape_start_class"]
config_pgr = config["pgr1"]
linkweb = config_pgr["link"]
nome_pgr = config_pgr["programma"].capitalize()
tipo_pgr = config_pgr["tipologia"].capitalize()


#####--- 1.1 Define website
base_url = linkweb
response = requests.get(base_url)
html_page = response.text
html_tree = BeautifulSoup(html_page)
html_tree = BeautifulSoup(html_page, 'html.parser')

#####--- 1.2 Define elements list to scrape
lista_pgr = []
lisa_type_durata  = []
lista_inizio_pgr = []

#####--- 1.3 Define classes
programmi = html_tree.find_all("span", class_ = scrape_pgr_class)
type_durata= html_tree.find_all("span", class_ = scrape_type_dur )
inizio_pgr= html_tree.find_all("div", class_ = scrape_start_class)

## 2. Scrape and create dataset

In [5]:
#####--- 2.0 Scrape programs list
for p in programmi:
    lista_pgr.append(p.string)

#####--- 2.1 Scrape programs types and duration
for o in type_durata:
    lisa_type_durata.append(o.string)

#####--- 2.2 Scrape starting time
for i in inizio_pgr:
    lista_inizio_pgr.append(i.contents[0])
    
df = pd.DataFrame(list(zip(lista_pgr,lisa_type_durata,lista_inizio_pgr)), 
                  columns = ["Programma","Durata","Orario inizio"])

## 3. Process dataframe

In [9]:
#####--- 3.0 Split Tipologia and Durata
df[["Tipologia","Durata"]] = df["Durata"].str.split(r"\(", expand = True)
df["Tipologia"] = df["Tipologia"].str.strip()
df["Durata"] = df["Durata"].apply(only_duration)
df["Durata"] = df["Durata"].astype(int)

#####--- 3.1 Set datetime format for Orario inizio
df["Orario inizio"] = df["Orario inizio"].astype("str")
df["Orario inizio"] = df["Orario inizio"].str.strip()
df["Orario inizio"] = pd.to_datetime(df['Orario inizio'])

#####--- 3.2 Create ending time
df['Orario fine'] = df_final["Orario inizio"]+ pd.TimedeltaIndex(df['Durata'], unit='m')

#####--- 3.3 Create hashtag variable
df["Hashtags"] = df["Programma"].str.lower()
df["Hashtags"] = df["Hashtags"].str.strip()

#####--- 3.4 Create Giorno settimana variable
df["Giorno settimana"] = df["Orario inizio"].dt.day_name(locale="it_IT")

## 4. Filter based on program and type

In [13]:
#####--- 4.0 Filter programs
df = df.loc[(df["Tipologia"] == tipo_pgr)&(df["Programma"] == nome_pgr)]
df.drop_duplicates(subset = ["Programma"], keep= "last", inplace = True)

#####--- 4.1 Drop some variables
df.drop(["Durata","Tipologia"], axis = 1,inplace  = True)

In [21]:
df.head()

,Programma,Orario inizio,Orario fine,Giorno settimana,Hashtags
17,Quarta repubblica,2021-02-08 21:25:00,2021-02-09 00:51:00,Lunedì,quartarepubblica
